In [1]:
import csv
import hazm as h
import pandas as pd

In [2]:
normalizer = h.Normalizer()
stemmer = h.Stemmer()
tokenizer = h.WordTokenizer()
lemmatizer=h.Lemmatizer()

In [12]:
df=pd.read_csv('books_train.csv')
df2=pd.read_csv('books_test.csv')
df3=pd.read_csv('sw.csv')
stop_words=df3.values.tolist()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2550 non-null   object
 1   description  2550 non-null   object
 2   categories   2550 non-null   object
dtypes: object(3)
memory usage: 59.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        450 non-null    object
 1   description  450 non-null    object
 2   categories   450 non-null    object
dtypes: object(3)
memory usage: 10.7+ KB


In [4]:

import string


def delete_digit_and_punctuation(input):
    output=[]
    punctuations = ['»','،','«',' ،', '، ',')','(','؛','-']
    for i in range(len(input)) :
        input[i] = lemmatizer.lemmatize(input[i])
        if ((input[i].isdigit()) or (input[i] in punctuations) or (input[i]in stop_words)) == False:
            output.append(input[i])
    return output

def virayesh(input):
    input = normalizer.remove_specials_chars(input)
    finalinput= tokenizer.tokenize(input)
    finalinput =delete_digit_and_punctuation(finalinput)

    return finalinput

df['title'] = df['title'].str.translate(str.maketrans('', '', string.punctuation))
df['title']=df['title'].apply(virayesh)
df['description']=df['description'].apply(virayesh)

df2['title'] = df2['title'].str.translate(str.maketrans('', '', string.punctuation))
df2['title']=df2['title'].apply(virayesh)
df2['description']=df2['description'].apply(virayesh)


In [5]:
df

,title,description,categories
0,"[ساختار, نظریه, جامعه‌شناس, ایران]","[ساختار, نظریه, جامعه‌شناس, ایران, نوشته, ابوا...",جامعه‌شناسی
1,"[جامعه, و, فرهنگ, کانادا]","[جامعه, و, فرهنگ, کانادا, از, مجموعه, کتاب, جا...",جامعه‌شناسی
2,"[پرسش, از, موعود]","[پرسش, مختلف, درباره, زندگی, و, شخصیت, امام, م...",کلیات اسلام
3,"[موج, دریا]","[موج, دریا, به, قلم, مهر, ماهوت, و, تصویرگر, ع...",داستان کودک و نوجوانان
4,"[پرسش, از, غرب]","[پرسش, از, غرب, به, قلم, دکتر, اسماعیل, شفیع, ...",جامعه‌شناسی
...,...,...,...
2545,"[قضاوت‌‏های, امیرمؤمنان, حضرت, علی, علیه‏‌السلام]","[آنچه, به, عنوان, قضاوت, امیرمؤمنان, حضرت, علی...",کلیات اسلام
2546,"[کرامات, از, مهدی, موعودعج]","[زینب, محمود, پژوهشگر, دین, #است, ., \r, در, ب...",کلیات اسلام
2547,"[انتقام, انتقام, انتقام]","[انتقام, انتقام, انتقام, داستان‌گونه‌ای, با, ز...",داستان کوتاه
2548,"[آن, مرد, آمد#آ]","[آن, مرد, آمد#آ, به, قلم, سیده‌فاطمه, موسوی, و...",داستان کودک و نوجوانان


In [6]:
Bow={'داستان کوتاه':{} ,'مدیریت و کسب و کار':{},'رمان':{},'کلیات اسلام':{},'داستان کودک و نوجوانان':{},'جامعه‌شناسی':{}}
Totall=[]
for i in df.index:
    cat =  df['categories'][i]
    for word in df['description'][i]:
        if word in Bow[cat]:
            Bow[cat][word]+=1
          #  Totall[cat]+=1
        else :
            Bow[cat][word] = 1 
           # Totall[cat]+=1


for i in df.index:
    cat =  df['categories'][i]
    for word in df['title'][i]:
        if word in Bow[cat]:
           # Totall[cat]+=1
            Bow[cat][word]+=1
        else :
            #Totall[cat]+=1
            Bow[cat][word] = 1 
    


In [7]:

for c in Bow :
    count = 0
    for w in Bow[c] : 
        count += Bow[c][w]
    for w in Bow[c] : 
        Bow[c][w] = Bow[c][w]/count



In [8]:
import math
categories = list(df2["categories"].unique())
df2["predicted_cat"] = -1
for i in df2.iterrows():
    probs = []
    for cat in categories:
        prob = 1
        for w in i[1]["title"]:
            p = 0
            if(w in Bow[cat]):
                p = Bow[cat][w]
            if (p == 0):
               p = 1 / 200000
            prob +=  math.log(p)
        for d in i[1]["description"]:
            p = 0
            if(d in Bow[cat]):
                p = Bow[cat][d]
            if (p == 0):
               p = 1 / 200000
            prob += math.log(p)
        probs.append(prob)
    index = probs.index(max(probs))
    df2.loc[i[0] , ["predicted_cat"]] = index
            


In [9]:
l = df2["categories"].to_list()
for i in range(len(l)):
    l[i] = categories.index(l[i])
df2["categories"] = l

In [10]:
len(df2[df2["categories"] == df2["predicted_cat"]])/450

0.8155555555555556